# Flood Adaptation: Minimal Model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as a simplified representation of household adaptation to flood events in a social network context. The model serves as a template for understanding the fundamental mechanics of flood adapdtation with a given social network.

This directory is structured as follows:
 - A Mesa Agent class `Households` is defined in `agents.py`
 - A Mesa Agent class `Government` is defined in `agents.py` (this agent currently does not perform any action)
 - A Mesa Model class `AdaptationModel` is defined in `model.py`
 - This notebook `demo.ipynb` provides a demonstration of the model and its visualization.

Each household agent in the model represents a household with attributes and behaviors that determine its response to flood events. The flood depth attribute assigned to each household is currently random for demonstration purposes. The decision by each household to adapt is also arbitrary. In a more comprehensive model, this could be replaced with more sophisticated decision-making algorithms.

The current implementation includes several simplifications and arbitrary choices, such as random flood depths and basic adaptation logic based on random thresholds. These aspects are designed for clarity and ease of understanding and should be replaced with realistic and sound choices.

You are encouraged to modify, expand, and customize the model. This might include integrating real-world data, implementing more complex decision-making processes for agents, or exploring different network topologies and their effects on social dynamics.

The visualisation provided is basic. Additional aspects can be added for visualization and more in-depth analysis.

_This notebook was tested on 2023-11-13 with Mesa version 2.1.4 on Python 3.12.0._

# Running the model and visualizing the model domain and social network over time

In [2]:
import random

from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx

# Determine the input values for your network and dynamic parameters and how they should be distributed throughout the population.
network_dynamics_dictionary = {
                # The social network structure that is used. Can currently be
                # "erdos_renyi", "barabasi_albert", "watts_strogatz", or "no_network"
                "network": 'watts_strogatz',
                "number_of_households": 50,  # number of household agents
                "probability_of_network_connection": 0.4,  # likeliness of edge being created between two nodes
                "number_of_edges": 3,  # number of edges for BA network
                "number_of_nearest_neighbours": 5,  # number of nearest neighbours for WS social network
                "flood_time_tick": [5],  # list of flood times on the tick
                "flood_severity_probability": (0.5, 1.2),  # Bounds between a Uniform distribution of the flood severity
                "seed": 1  # The seed used to generate pseudo random numbers
            }

# Determine the input values for your agent's attributes and how they should be distributed throughout the population.
attribute_dictionary = {
                "wealth": [0.2, 'UI', (0, 3)],
                "has_child": [0.2, 'B', (0.2)],
                "house_size": [0.05, 'UI', (0, 2)],
                "house_type": [0.05, 'UI', (0, 1)],
                "education_level": [0.05, 'UI', (0, 3)],
                "social_preference": [0.2, 'U', (-1, 1)],
                "age": [0.2, 'N', (33.4, 5)]
            }

# Defining the standard values for agent interaction dynamics in case none are provided.
agent_interaction_dictionary = {
                "household_tolerance": 0.15,  # the tolerance level for each agent
                "bias_change_per_tick": 0.2,  # Bias change per tick when an agent when it's influenced by its network
                "probability_positive_bias_change": 0.5,  # Probability that agent changes it's bias positively
                "probability_negative_bias_change": 0.1,  # Probability that agent changes it's bias negatively
                "adaption_threshold": 0.7  # Threshold of bias an agent needs to adapt
            }

# Initialize the Adaptation Model with 50 household agents.
model = AdaptationModel(network_dynamics_dictionary=network_dynamics_dictionary, attribute_dictionary=attribute_dictionary, agent_interaction_dictionary= agent_interaction_dictionary)
# flood_map_choice can be "harvey", "100yr", or "500yr"

# Calculate positions of nodes for the network plot.
# The spring_layout function positions nodes using a force-directed algorithm,
# which helps visualize the structure of the social network.
pos = nx.spring_layout(model.G)

# Define a function to plot agents on the network.
# This function takes a matplotlib axes object and the model as inputs.
# def plot_network(ax, model):
#     # Clear the current axes.
#     ax.clear()
#     # Determine the color of each node (agent) based on their adaptation status.
#     colors = ['blue' if agent.is_adapted else 'red' for agent in model.schedule.agents]
#     # Draw the network with node colors and labels.
#     nx.draw(model.G, pos, node_color=colors, with_labels=True, ax=ax)
#     # Set the title of the plot with the current step number.
#     ax.set_title(f"Social Network State at Step {model.schedule.steps}", fontsize=12)

# Generate the initial plots at step 0.
# Plot the spatial distribution of agents. This is a function written in the model.py
# model.plot_model_domain_with_agents()

# Plot the initial state of the social network.
# fig, ax = plt.subplots(figsize=(7, 7))
# plot_network(ax, model)
# plt.show()

# Run the model for 20 steps and generate plots every 5 steps.
for step in range(80):
    model.step()
# 
#     # Every 5 steps, generate and display plots for both the spatial distribution and network.
#     # Note the first step is step 0, so the plots will be generated at steps 4, 9, 14, and 19, which are the 5th, 10th, 15th, and 20th steps.
#     if (step + 1) % 10 == 0:
#         # Plot for the spatial map showing agent locations and adaptation status.
#         plt.figure(figsize=(10, 6))
#         model.plot_model_domain_with_agents()
# 
#         # Plot for the social network showing connections and adaptation statuses.
#         fig, ax = plt.subplots(figsize=(7, 7))
#         plot_network(ax, model)
#         plt.show()

step: 0 : 8 adapted with a bias of 0 and a estimated damage of 0.7317721107910228!
step: 0 : 49 adapted with a bias of 0 and a estimated damage of 0.8130381003936059!
step: 0 : 3 adapted with a bias of 0 and a estimated damage of 1!
step: 0 : 23 adapted with a bias of 0 and a estimated damage of 0.9397987376666705!
step: 0 : 6 adapted with a bias of 0.2 and a estimated damage of 0.9006906221541626!
step: 0 : 26 adapted with a bias of 0 and a estimated damage of 0.9070094801567291!
step: 0 : 43 adapted with a bias of 0 and a estimated damage of 0.8094178153612082!
step: 0 : 48 adapted with a bias of 0 and a estimated damage of 0.7779743412871583!
step: 0 : 22 adapted with a bias of 0 and a estimated damage of 0.7606385065846636!
step: 0 : 13 adapted with a bias of 0 and a estimated damage of 0.7989886084436189!
step: 3 : 46 adapted with a bias of 0.4 and a estimated damage of 0.43531322756115387!
step: 3 : 17 adapted with a bias of 0.2 and a estimated damage of 0.6563603657345186!
step:

In [17]:
agent_data = model.datacollector.get_agent_vars_dataframe()
len([i for i in model.schedule.agents if i.has_child])
agent_data

# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a estimated flood damage of {round(agent.flood_damage_estimated, 3)} and is adapted is {agent.is_adapted} and has identity: {agent.identity}\n')
#     
# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a impacted bias because of flooding {round(agent.actual_flood_impact_on_bias, 3)}')

    

FloodDepthEstimated  FloodDamageEstimated  FloodDepthActual  \
Step AgentID                                                                
0    0                   0.914719              0.632736          0.000000   
     1                   0.828465              0.615444          0.000000   
     2                   1.477482              0.716453          0.000000   
     3                   0.575500              0.551831          0.000000   
     4                   0.000000              0.000000          0.000000   
...                           ...                   ...               ...   
79   45                  0.000000              0.000000          0.000000   
     46                  0.272741              0.421454          1.190511   
     47                  1.684425              0.739341          7.279849   
     48                  4.126419              0.895780         11.851432   
     49                  0.559410              0.546880          2.621033   

              FloodDamageActual  IsAdapted  identity  FriendsCount  \
Step AgentID                                                           
0    0                 0.000000      False    0.137915             3   
     1                 0.000000      False    0.238371             6   
     2                 0.000000      False    0.345650             2   
     3                 0.000000      False    0.487069             4   
     4                 0.000000      False    0.721177             6   
...                         ...        ...         ...           ...   
79   45                0.000000       True    0.502418             4   
     46                0.678747       True    0.885888             3   
     47                1.000000       True    1.124265             3   
     48                1.000000       True    0.938995             4   
     49                0.816539       True    0.402953             5   

                                                   location  HasChild  \
Step AgentID                                                            
0    0         POINT (219023.22382247538 3329850.249745885)     False   
     1           POINT (285073.881667405 3302426.377471865)     False   
     2        POINT (256732.87332841553 3345792.6517015398)     False   
     3        POINT (222614.99221068708 3303042.9055755553)     False   
     4        POINT (260072.42832994874 3314393.3591049607)     False   
...                                                     ...       ...   
79   45        POINT (254850.17420973376 3339061.844511192)     False   
     46        POINT (240641.2956753731 3288497.9238458956)     False   
     47       POINT (261288.78367108776 3289785.8242065767)      True   
     48        POINT (231088.2976510692 3302210.5498792124)      True   
     49       POINT (222131.39951582125 3293873.0438353326)      True   

              AgentBias  
Step AgentID             
0    0              0.0  
     1              0.0  
     2              0.0  
     3              0.0  
     4              0.0  
...                 ...  
79   45             6.4  
     46             8.0  
     47             0.0  
     48             7.8  
     49            14.8  

[4000 rows x 10 columns]

In [18]:
# sum([i.general_bias_in_network for i in model.schedule.agents])/len(model.schedule.agents)

max([i.actual_flood_impact_on_bias for i in model.schedule.agents])

0.7799932482854881

In [38]:
model_data = model.datacollector.get_model_vars_dataframe()
model_data

,total_adapted_households
0,0
1,8
2,12
3,12
4,14
...,...
75,20
76,20
77,20
78,20
